# TELL Runs for the National Transmission Planning (NTP) Project

This notebook executes the initial set of runs of the TELL model for the NTP project.

In [1]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell
import sklearn

import pandas as pd
import matplotlib.pyplot as plt

from tell.package_data import get_ba_abbreviations


## Set the Directory Structure

In [2]:
# Identify the top-level directory and the subdirectory where the data will be stored:
current_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs'
tell_data_dir = os.path.join(current_dir, r'tell_data')

# If the "tell_data_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_data_dir):
   os.makedirs(tell_data_dir)


## Run the MLP Models

In [ ]:
# Generate a list of BA abbreviations to process:
ba_abbrev_list = tell.get_balancing_authority_to_model_dict().keys()

scenario_to_process = 'historic'

# Run the MLP prediction step for the list of BAs using parallel processing streams:
for year_to_process in range(2005,2007,1):
    pdf = tell.predict_batch(target_region_list = ba_abbrev_list,
                             year = year_to_process,
                             data_dir = os.path.join(tell_data_dir, r'wrf_to_tell_data', scenario_to_process),
                             datetime_field_name = 'Time_UTC',
                             save_prediction = True,
                             prediction_output_directory = os.path.join(tell_data_dir, r'outputs', r'mlp_output', scenario_to_process),
                             n_jobs = -1)


## Run the Forward Execution

In [3]:
# Run the TELL model forward in time for a given year in five year increments out to 2100:
for year in range(2006,2007,1):
    summary_df, ba_time_series_df, state_time_series_df = tell.execute_forward(year_to_process = str(year),
                                                                               gcam_target_year = str(year), 
                                                                               scenario_to_process = 'historic',
                                                                               data_output_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/tell_output',
                                                                               gcam_usa_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/gcamusa_data',
                                                                               map_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/ba_service_territory_data',
                                                                               mlp_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/mlp_output',
                                                                               pop_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/population_data',
                                                                               save_county_data = False)


Start time =  2022-12-13 18:00:55.158418
End time =  2022-12-13 18:02:04.376362
Elapsed time =  0:01:09.218030


In [ ]:
import glob
import datetime

import numpy as np
import pandas as pd

from pandas import DataFrame
from scipy import interpolate
from tell.states_fips_function import state_metadata_from_state_abbreviation


In [ ]:
year_to_process = '2006'
gcam_target_year = '2006' 
scenario_to_process = 'historic'
data_output_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/tell_output'
gcam_usa_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/gcamusa_data'
map_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/ba_service_territory_data'
mlp_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/mlp_output'
pop_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/population_data'
save_county_data = False


In [ ]:
# Load in the sample GCAM-USA output file and subset the data to only the "year_to_process":
gcam_usa_df = tell.extract_gcam_usa_loads(scenario_to_process = scenario_to_process, 
                                          filename = (os.path.join(gcam_usa_input_dir, 'gcamDataTable_aggParam.csv')))
    
gcam_usa_df = gcam_usa_df[gcam_usa_df['Year'] == int(gcam_target_year)]

# Load in the most recent (i.e., 2019) BA service territory mapping file:
ba_mapping_df = pd.read_csv((os.path.join(map_input_dir, 'ba_service_territory_2019.csv')), index_col=None, header=0)

# Read in the sample population projection dataset for the scenario being processed:
population_df = tell.process_population_scenario(scenario_to_process, pop_input_dir)

# Subset to only the year being processed:
if (scenario_to_process == 'historic') and (int(year_to_process) < 2000):
    population_df = population_df.loc[(population_df['Year'] == 2000)]
else:
    population_df = population_df.loc[(population_df['Year'] == int(year_to_process))]

population_df
